In [1]:
import pandas as pd
from pybaseball import playerid_lookup

In [2]:
players = pd.read_csv('../../data/output/cleaned_knuckleballers.csv')

players['first'] = players['Name'].apply(lambda x: x.split(' ')[0])
players['last'] = players['Name'].apply(lambda x: x.split(' ')[-1])

players['career_start'] = players['Career'].apply(lambda x: int(x[:4]))

In [3]:
# inspect for anyone with different convention
pd.set_option('display.max_rows', None)
players

,Name,Throws,Career,first,last,career_start
0,Al Papai,R,1948-55,Al,Papai,1948
1,Andy Karl,R,1943-47,Andy,Karl,1943
2,Art Evans,L,1932,Art,Evans,1932
3,Aubrey Gatewood,R,1963-70,Aubrey,Gatewood,1963
4,Barney Schultz,R,1955-65,Barney,Schultz,1955
5,Ben Flowers,R,1951-56,Ben,Flowers,1951
6,Bob Humphreys,R,1962-70,Bob,Humphreys,1962
7,Bob Moorhead,R,1962-65,Bob,Moorhead,1962
8,Bob Purkey,R,1954-66,Bob,Purkey,1954
9,Bob Tiefenauer,R,1952-68,Bob,Tiefenauer,1952


In [4]:

row = players.to_dict('records')[0]
id_data = playerid_lookup(row['last'], row['first'], fuzzy=True)


Gathering player lookup table. This may take a moment.


In [5]:
matches = pd.DataFrame()
missing = []
for index, row in players.iterrows(): 
    id_data = playerid_lookup(row['last'], row['first'], fuzzy=True)
    possible_matches = id_data[id_data.mlb_played_first==row['career_start']].copy()
    if len(possible_matches) > 0: 
        match = possible_matches.head(1).copy()
        match['name_original'] = row['Name']
        match['career_span_original'] = row['Career']
        matches = pd.concat([matches, match])
    else: 
        # testing found that any missing players had only one fuzzy match that looked correct -> use that
        match = id_data.head(1).copy()
        match['name_original'] = row['Name']
        match['career_span_original'] = row['Career']
        matches = pd.concat([matches, match])

No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.
No identically matched names found! Returning the 5 most similar names.


In [6]:
# # spot check
# matches[['name_last','name_first','name_original','mlb_played_first','mlb_played_last','career_span_original']]

In [7]:
# # spot check
# ix = 4
# playerid_lookup(missing[ix][0], missing[ix][1], fuzzy=True)

In [8]:
assert len(matches) == len(players)

In [9]:
matches.to_csv('../../data/input/cleaned_knuckleballers_with_ids.csv', index=False)